In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from google.colab import files
import time
import struct
import PIL

In [3]:
import torchvision.models as models
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
gpu_boole = torch.cuda.is_available()
print(gpu_boole)

True


In [ ]:
#input_data_dir = "/content/drive/My Drive/tiny-imagenet-200/"

In [5]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip tiny-imagenet-200.zip

'''
current="$(pwd)/tiny-imagenet-200"

# training data
cd $current/train
for DIR in $(ls); do
   cd $DIR
   rm *.txt
   mv images/* .
   rm -r images
   cd ..
done

# validation data
cd $current/val
annotate_file="val_annotations.txt"
length=$(cat $annotate_file | wc -l)
for i in $(seq 1 $length); do
    # fetch i th line
    line=$(sed -n ${i}p $annotate_file)
    # get file name and directory name
    file=$(echo $line | cut -f1 -d" " )
    directory=$(echo $line | cut -f2 -d" ")
    mkdir -p $directory
    mv images/$file $directory
done
rm -r images
echo "done"
'''

Streaming output truncated to the last 5000 lines.
  inflating: tiny-imagenet-200/val/images/val_3979.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3963.JPEG  
  inflating: tiny-imagenet-200/val/images/val_7199.JPEG  
  inflating: tiny-imagenet-200/val/images/val_2752.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9687.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9407.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3603.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3412.JPEG  
  inflating: tiny-imagenet-200/val/images/val_6982.JPEG  
  inflating: tiny-imagenet-200/val/images/val_8496.JPEG  
  inflating: tiny-imagenet-200/val/images/val_7332.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9241.JPEG  
  inflating: tiny-imagenet-200/val/images/val_4196.JPEG  
  inflating: tiny-imagenet-200/val/images/val_5980.JPEG  
  inflating: tiny-imagenet-200/val/images/val_6697.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9969.JPEG  
  inflating: tiny-ima

'\ncurrent="$(pwd)/tiny-imagenet-200"\n\n# training data\ncd $current/train\nfor DIR in $(ls); do\n   cd $DIR\n   rm *.txt\n   mv images/* .\n   rm -r images\n   cd ..\ndone\n\n# validation data\ncd $current/val\nannotate_file="val_annotations.txt"\nlength=$(cat $annotate_file | wc -l)\nfor i in $(seq 1 $length); do\n    # fetch i th line\n    line=$(sed -n ${i}p $annotate_file)\n    # get file name and directory name\n    file=$(echo $line | cut -f1 -d" " )\n    directory=$(echo $line | cut -f2 -d" ")\n    mkdir -p $directory\n    mv images/$file $directory\ndone\nrm -r images\necho "done"\n'

In [6]:
#ImageNet dataloading taken from 
#https://github.com/tjmoon0104/pytorch-tiny-imagenet/blob/master/AlexNet.ipynb
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
    ])
}

data_dir = 'tiny-imagenet-200/'
num_workers = {
    'train' : 100,
    'val'   : 0,
    'test'  : 0
}
image_datasets = {x: dataset.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val','test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=100,
                                             shuffle=True, num_workers=num_workers[x])
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

In [ ]:
print(dataset_sizes)

for i, data in enumerate(dataloaders['train']):
  images, labels = data

In [8]:
netparam = models.resnet34(pretrained=True)
netparam.fc.out_features = 200
print(netparam.parameters)
optimizer = torch.optim.Adam(netparam.parameters(), 0.001)
loss_metric = torch.nn.CrossEntropyLoss()
if gpu_boole:
 netparam = netparam.cuda()

<bound method Module.parameters of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [12]:
def train_eval(netparam, verbose = 1):
    correct = 0
    total = 0
    loss_sum = 0
    for images, labels in dataloaders['train']:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()
        outputs = netparam(images)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        total += labels.size(0)
        correct += (predicted.float() == labels.float()).sum()
        loss_sum += loss_metric(outputs,labels).detach()
        
    if verbose:
        print('Train accuracy: %f %%' % (100.0 * correct / total))
        print('Train loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return 100.0 * correct / total, loss_sum.cpu().data.numpy().item() / total
    
def test_eval(netparam, verbose = 1):
    correct = 0
    total = 0
    loss_sum = 0
    for images, labels in dataloaders['val']:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()    
        outputs = netparam(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.float() == labels.float()).sum()
        loss_sum += loss_metric(outputs,labels).detach()

    if verbose:
        print('Test accuracy: %f %%' % (100.0 * correct / total))
        print('Test loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return 100.0 * correct / total, loss_sum.cpu().data.numpy().item() / total

def validation_eval(netparam, verbose = 1):
    correct = 0
    total = 0
    loss_sum = 0
    for images, labels in dataloaders['test']:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()
        outputs = netparam(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.float() == labels.float()).sum()
        loss_sum += loss_metric(outputs,labels).detach()

    if verbose:
        print('Validation accuracy: %f %%' % (100.0 * correct / total))
        print('Validation loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return 100.0 * correct / total, loss_sum.cpu().data.numpy().item() / total

In [14]:
epochs = 10

#defining batch train loss recording arrays for later visualization/plotting:
loss_batch_store = []
train_accuracy_scores = []
test_accuracy_scores = []
validation_accuracy_scores = []
training_loss = []
epoch_list = [i for i in range(1, epochs + 1)]
epoch_loss = []
maxvalacc = 0
print("Starting Training")
#training loop:
for epoch in range(epochs):
  time1 = time.time() #timekeeping
  for i, (x, y) in enumerate(dataloaders['train']):
    if (i % 100 ==0 ):
      print(i)
    if gpu_boole:
      x = x.cuda()
      y = y.cuda()
    if i > 0 or epoch > 0:
      optimizer.zero_grad()
    outputs = netparam.forward(x)
    loss = loss_metric(outputs,y)
    loss.backward()
    if i > 0 or epoch > 0:
      loss_batch_store.append(loss.cpu().data.numpy().item())
    optimizer.step()
    epoch_loss.append(loss.detach())
    

  print("Epoch",epoch+1,':')
  valid_perc, valid_loss = validation_eval(netparam)
  #if valid_perc > maxvalacc:
   # maxvalacc = valid_perc
    #torch.save(netparam.state_dict(), path) 
  validation_accuracy_scores.append(valid_perc)
  time2 = time.time() #timekeeping
  print('Elapsed time for epoch:',time2 - time1,'s')
  print('ETA of completion:',(time2 - time1)*(epochs - epoch - 1)/60,'minutes')
  print()

## Plotting batch-wise train loss curve:
#plt.plot(epoch_list, training_loss, '-o', label = 'train_loss', color = 'blue')
#plt.xlabel('Epoch Number')
#plt.ylabel('Training Loss at Given Epoch')
#plt.legend()
#plt.show()
#plt.plot(epoch_list, train_accuracy_scores, color = 'blue')
#plt.plot(epoch_list, test_accuracy_scores, color = 'blue')
#plt.plot(epoch_list, validation_accuracy_scores, color = 'blue')
#plt.xlabel('Epoch Number')
#plt.ylabel('Accuracy in Percent')
test_perc_a, test_loss_a = test_eval(netparam)

Starting Training
0
100
200
300
400
500
600
700
800
900
Epoch 1 :
Validation accuracy: 1.140000 %
Validation loss: 0.095106
Elapsed time for epoch: 74.59289813041687 s
ETA of completion: 11.18893471956253 minutes

0
100
200
300
400
500
600
700
800
900
Epoch 2 :
Validation accuracy: 0.610000 %
Validation loss: 0.109776
Elapsed time for epoch: 73.75708675384521 s
ETA of completion: 9.834278233846028 minutes

0
100
200
300
400
500
600
700
800
900
Epoch 3 :
Validation accuracy: 0.420000 %
Validation loss: 0.120361
Elapsed time for epoch: 74.51844263076782 s
ETA of completion: 8.693818306922912 minutes

0
100
200
300
400
500
600
700
800
900
Epoch 4 :
Validation accuracy: 0.580000 %
Validation loss: 0.126018
Elapsed time for epoch: 73.83095288276672 s
ETA of completion: 7.383095288276673 minutes

0
100
200
300
400
500
600
700
800
900
Epoch 5 :
Validation accuracy: 0.400000 %
Validation loss: 0.144953
Elapsed time for epoch: 71.9823489189148 s
ETA of completion: 5.998529076576233 minutes



KeyboardInterrupt: ignored